In [2]:
import cv2
import os
import csv
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import numpy as np
def draw_hand_preview(base_img,x, y, z, 
                      thumb, fore, index, ring, little, 
                      thumb2, fore2, index2, ring2, little2,
                      gs1, gs2):
    """
    Dibuja ambas manos en una vista previa según los sliders y booleanos.
    gs1 = mano izquierda activa
    gs2 = mano derecha activa
    """
    canvas = base_img.copy()

    scale = 0.5 + (z / 100)  # Escala según Z

    # ---------------------------
    # Mano izquierda (gs1)
    # ---------------------------
    if gs1:
        cx = int(150 + x * 2)
        cy = int(400 - y * 3)
        radius = int(40 * scale)
        cv2.circle(canvas, (cx, cy), radius, (0, 150, 255), 3)

        fingers = {
            "thumb": thumb,
            "fore": fore,
            "index": index,
            "ring": ring,
            "little": little
        }
        offsets = {
            "thumb": -60,
            "fore": -20,
            "index": 0,
            "ring": 20,
            "little": 40
        }

        for name, flex in fingers.items():
            ox = offsets[name]
            base_x = cx + int(ox * scale)
            base_y = cy - int(radius * 0.2)
            length = int(90 * scale)

            # Pulgar apunta hacia adentro (mano izquierda)
            if name == "thumb":
                angle = np.radians(-90 * (flex / 100))
            else:
                angle = np.radians(90 * (flex / 100))

            tip_x = int(base_x + length * np.sin(angle))
            tip_y = int(base_y - length * np.cos(angle))
            cv2.line(canvas, (base_x, base_y), (tip_x, tip_y), (50, 50, 50), 4)
            cv2.circle(canvas, (tip_x, tip_y), 6, (0, 0, 0), -1)

    # ---------------------------
    # Mano derecha (gs2)
    # ---------------------------
    if gs2:
        cx = int(350 + x * 2)
        cy = int(400 - y * 3)
        radius = int(40 * scale)
        cv2.circle(canvas, (cx, cy), radius, (255, 100, 0), 3)

        fingers = {
            "thumb": thumb2,
            "fore": fore2,
            "index": index2,
            "ring": ring2,
            "little": little2
        }
        offsets = {
            # Invertimos el offset horizontal del pulgar
            "thumb": 60,
            "fore": -20,
            "index": 0,
            "ring": 20,
            "little": 40
        }

        for name, flex in fingers.items():
            ox = offsets[name]
            base_x = cx + int(ox * scale)
            base_y = cy - int(radius * 0.2)
            length = int(90 * scale)

            # Pulgar apunta hacia afuera (mano derecha)
            if name == "thumb":
                angle = np.radians(90 * (flex / 100))
            else:
                angle = np.radians(-90 * (flex / 100))

            tip_x = int(base_x + length * np.sin(angle))
            tip_y = int(base_y - length * np.cos(angle))
            cv2.line(canvas, (base_x, base_y), (tip_x, tip_y), (50, 50, 50), 4)
            cv2.circle(canvas, (tip_x, tip_y), 6, (0, 0, 0), -1)

    return canvas


# -----------------------------
# CONFIG
# -----------------------------
DATASET_FOLDER = "Image"        # carpeta donde están las imágenes
CSV_OUTPUT = "dataset_labeled.csv"
WINDOW_SIZE = (900, 700)
SLIDER_WIDTH = 250

# -----------------------------
# CSV HEADER
# -----------------------------
HEADER = [
    "image_path",
    "x","y","z",
    "thumb","fore","index","ring","little",
    "thumb2","fore2","index2","ring2","little2",
    "keycode","gs1","gs2",
    "sign",
    "label"
]

# -----------------------------
# Listar imágenes
# -----------------------------
def list_images(dataset_path):
    imgs = []
    for root, dirs, files in os.walk(dataset_path):
        for f in files:
            if f.lower().endswith((".png",".jpg",".jpeg",".bmp")):
                imgs.append(os.path.join(root, f))
    return imgs

# -----------------------------
# Dibujar rectángulo en OpenCV
# -----------------------------
drawing = False
ix, iy = -1, -1
rx1 = ry1 = rx2 = ry2 = -1

def draw_rect(event, x, y, flags, param):
    global ix, iy, drawing, rx1, ry1, rx2, ry2, img_copy

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img_copy = img_show.copy()
        cv2.rectangle(img_copy, (ix, iy), (x, y), (0,255,0), 2)
        cv2.imshow("Etiqueta la imagen", img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rx1, ry1, rx2, ry2 = ix, iy, x, y
        img_copy = img_show.copy()
        cv2.rectangle(img_copy, (rx1, ry1), (rx2, ry2), (0,255,0), 2)
        cv2.imshow("Etiqueta la imagen", img_copy)

def update_slider_value(self, varname):
    value = self.sliders[varname].get()
    self.slider_labels[varname].config(text=str(value))
    self.update_preview()
    
# -----------------------------
# Aplicación Tkinter
# -----------------------------
class LabelingApp:
    def __init__(self, master, images):
        self.master = master
        self.images = images
        self.index = 0

        master.title("Etiquetador de Dataset")
        master.geometry(f"{WINDOW_SIZE[0]}x{WINDOW_SIZE[1]}")

        self.create_widgets()
        self.load_next_image()
        
    def update_slider_value(self, varname):
        value = self.sliders[varname].get()
        self.slider_labels[varname].config(text=str(value))
        self.update_preview()
        

    def create_widgets(self):
        self.frame = ttk.Frame(self.master)
        self.frame.pack(pady=10)

        # Nombre archivo
        self.label_path = ttk.Label(self.frame, text="Imagen:")
        self.label_path.grid(row=0, column=0, columnspan=3, pady=5)

        # Sliders
        self.sliders = {}
        self.slider_labels = {}

        slider_defs = {
            "x": 100, "y": 50, "z": 100,
            "thumb": 100, "fore": 100, "index": 100, "ring": 100, "little": 100,
            "keycode": 10, "gs1": 1, "gs2": 1, "sign": 10
        }
        
        slider_defs2 = {
            "thumb2": 100, "fore2": 100, "index2": 100, "ring2": 100, "little2": 100,
            "gs1": 1, "gs2": 1
        }

        r = 1

        for var, maxv in slider_defs.items():
            ttk.Label(self.frame, text=f"{var.upper()}:").grid(row=r, column=0, padx=5, sticky="w")

            # Sliders con valores negativos para X e Y
            if var == "x":
                slider = tk.Scale(self.frame, from_=-150, to=150, orient=tk.HORIZONTAL,
                                length=SLIDER_WIDTH, command=lambda v, name=var: self.update_slider_value(name))
            elif var == "y":
                slider = tk.Scale(self.frame, from_=-100, to=100, orient=tk.HORIZONTAL,
                                length=SLIDER_WIDTH, command=lambda v, name=var: self.update_slider_value(name))
            else:
                slider = tk.Scale(self.frame, from_=0, to=maxv, orient=tk.HORIZONTAL,
                                length=SLIDER_WIDTH, command=lambda v, name=var: self.update_slider_value(name))
            slider.grid(row=r, column=1, padx=5)

            value_label = ttk.Label(self.frame, text="0")
            value_label.grid(row=r, column=2, padx=5)

            self.sliders[var] = slider
            self.slider_labels[var] = value_label
            r += 1

        # Sliders de la segunda mano
        for var, maxv in slider_defs2.items():
            ttk.Label(self.frame, text=f"{var.upper()}:").grid(row=r, column=0, padx=5, sticky="w")
            slider = tk.Scale(self.frame, from_=0, to=maxv, orient=tk.HORIZONTAL,
                            length=SLIDER_WIDTH, command=lambda v, name=var: self.update_slider_value(name))
            slider.grid(row=r, column=1, padx=5)
            value_label = ttk.Label(self.frame, text="0")
            value_label.grid(row=r, column=2, padx=5)
            self.sliders[var] = slider
            self.slider_labels[var] = value_label
            r += 1

        # Campo de etiqueta final
        ttk.Label(self.frame, text="Clase / Etiqueta (ej: A, B, C):").grid(row=r, column=0, pady=10)
        self.entry_label = ttk.Entry(self.frame)
        self.entry_label.grid(row=r, column=1)
        r += 1

        # Botón guardar
        self.btn_save = ttk.Button(self.frame, text="Guardar y siguiente", command=self.save_and_next)
        self.btn_save.grid(row=r, column=0, columnspan=3, pady=15)

        # Ajustar ventana al alto máximo de la pantalla
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()
        self.master.geometry(f"{WINDOW_SIZE[0]}x{screen_height}")


    # Cargar imagen siguiente
    def load_next_image(self):
        global img_show, img_copy

        if self.index >= len(self.images):
            messagebox.showinfo("FIN", "Todas las imágenes han sido etiquetadas.")
            self.master.quit()
            return

        self.current_img_path = self.images[self.index]
        self.label_path.config(text=f"Imagen: {self.current_img_path}")

        img_show = cv2.imread(self.current_img_path)
        img_copy = img_show.copy()
        cv2.namedWindow("Etiqueta la imagen")
        cv2.setMouseCallback("Etiqueta la imagen", draw_rect)
        cv2.imshow("Etiqueta la imagen", img_copy)
        self.update_preview()


    # Guardar línea en el CSV
    def save_and_next(self):
        data = [
            self.current_img_path,
            self.sliders["x"].get(),
            self.sliders["y"].get(),
            self.sliders["z"].get(),
            self.sliders["thumb"].get(),
            self.sliders["fore"].get(),
            self.sliders["index"].get(),
            self.sliders["ring"].get(),
            self.sliders["little"].get(),
            self.sliders["thumb2"].get(),
            self.sliders["fore2"].get(),
            self.sliders["index2"].get(),
            self.sliders["ring2"].get(),
            self.sliders["little2"].get(),
            self.sliders["keycode"].get(),
            self.sliders["gs1"].get(),
            self.sliders["gs2"].get(),
            self.sliders["sign"].get(),
            self.entry_label.get()
        ]


        exists = os.path.isfile(CSV_OUTPUT)
        with open(CSV_OUTPUT, "a", newline="") as f:
            writer = csv.writer(f)
            if not exists:
                writer.writerow(HEADER)
            writer.writerow(data)

        # Reset cuadro
        global rx1, ry1, rx2, ry2
        rx1 = ry1 = rx2 = ry2 = -1

        # Avanzar
        self.index += 1
        self.load_next_image()

    def update_preview(self):
        
        global img_copy

        img_copy = draw_hand_preview(
            img_show,  # la imagen original
            self.sliders["x"].get(),
            self.sliders["y"].get(),
            self.sliders["z"].get(),

            self.sliders["thumb"].get(),
            self.sliders["fore"].get(),
            self.sliders["index"].get(),
            self.sliders["ring"].get(),
            self.sliders["little"].get(),

            self.sliders["thumb2"].get(),
            self.sliders["fore2"].get(),
            self.sliders["index2"].get(),
            self.sliders["ring2"].get(),
            self.sliders["little2"].get(),

            self.sliders["gs1"].get(),
            self.sliders["gs2"].get()
        )

        cv2.imshow("Etiqueta la imagen", img_copy)

    


# -----------------------------
# MAIN
# -----------------------------
if __name__ == "__main__":
    images = list_images(DATASET_FOLDER)
    if len(images) == 0:
        print("No hay imágenes en la carpeta dataset/")
        exit()

    root = tk.Tk()
    app = LabelingApp(root, images)
    root.mainloop()

    cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

Con esto esta todo etiquetado

In [ ]:
!pip install opencv-python

In [ ]:
#Agregamos variables de giro que se me olvidaron

rotation_rules = {
    "B":  {"giroX": 90,  "giroZ": 180},
    "C":  {"giroZ": 90},
    "D":  {"giroZ": 90},
    "F":  {"giroZ": 70},
    "G":  {"giroZ": 180},   # marcadas como F pero rotación de G
    "J":  {"giroZ": 180},
    "K":  {"giroZ": 90},
    "M":  {"giroX": 180, "giroZ": 180},
    "N":  {"giroX": 180, "giroZ": 180},
    "O":  {"giroZ": 90},
    "Q":  {"giroZ": 180},
    "S":  {"giroZ": 90},
    "X":  {"giroZ": 90}
}


# para facilitar el proceso lo hice plano, ahora agregamos ruido


import pandas as pd
import numpy as np
import random

# -----------------------------
# CONFIGURACIÓN
# -----------------------------

INPUT_CSV = "dataset_labeled.csv"
OUTPUT_CSV = "dataset_with_noise.csv"

# Cantidad de ruido
RUIDO_POS = 5        # +/- pixeles en x,y,z
RUIDO_FINGER = 3     # +/- en flexión dedos
RUIDO_KEYCODE = 0    # opcional
RUIDO_SIGN = 0       # opcional

# Reglas de rotación por letra
rotation_rules = {
    "B":  {"giroX": 90,  "giroZ": 180},
    "C":  {"giroZ": 90},
    "D":  {"giroZ": 90},
    "F":  {"giroZ": 70},
    "G":  {"giroZ": 180},
    "J":  {"giroZ": 180},
    "K":  {"giroZ": 90},
    "M":  {"giroX": 180, "giroZ": 180},
    "N":  {"giroX": 180, "giroZ": 180},
    "O":  {"giroZ": 90},
    "Q":  {"giroZ": 180},
    "S":  {"giroZ": 90},
    "X":  {"giroZ": 90}
}


# -----------------------------
# FUNCIÓN PARA AGREGAR RUIDO
# -----------------------------
def add_noise(value, amount, minv=None, maxv=None):
    noisy = value + random.randint(-amount, amount)
    if minv is not None:
        noisy = max(minv, noisy)
    if maxv is not None:
        noisy = min(maxv, noisy)
    return noisy


# -----------------------------
# PROCESAR CSV
# -----------------------------
df = pd.read_csv(INPUT_CSV)

# Agregar columnas nuevas
df["giroX"] = 0
df["giroY"] = 0
df["giroZ"] = 0

# Columnas que llevarán ruido
pos_cols = ["x", "y", "z"]
finger_cols = ["thumb","fore","index","ring","little",
               "thumb2","fore2","index2","ring2","little2"]

print("Procesando dataset...")

for i in range(len(df)):
    row = df.loc[i]

    # -----------------------------
    # APLICAR RUIDO A POSICIONES
    # -----------------------------
    for p in pos_cols:
        df.loc[i, p] = add_noise(row[p], RUIDO_POS)

    # -----------------------------
    # APLICAR RUIDO A DEDOS
    # -----------------------------
    for f in finger_cols:
        df.loc[i, f] = add_noise(row[f], RUIDO_FINGER, 0, 100)

    # -----------------------------
    # RUIDO OPCIONAL EN keycode y sign
    # -----------------------------
    df.loc[i, "keycode"] = add_noise(row["keycode"], RUIDO_KEYCODE)
    df.loc[i, "sign"]    = add_noise(row["sign"], RUIDO_SIGN)

    # -----------------------------
    # AGREGAR ROTACIÓN SEGÚN LA LETRA
    # -----------------------------
    label = row["label"].upper()

    if label in rotation_rules:
        rules = rotation_rules[label]
        df.loc[i, "giroX"] = rules.get("giroX", 0)
        df.loc[i, "giroZ"] = rules.get("giroZ", 0)
        df.loc[i, "giroY"] = rules.get("giroY", 0)
    else:
        df.loc[i, ["giroX","giroY","giroZ"]] = 0


# -----------------------------
# GUARDAR NUEVO CSV
# -----------------------------
df.to_csv(OUTPUT_CSV, index=False)
print("Listo! Archivo generado:", OUTPUT_CSV)



Procesando dataset...
Listo! Archivo generado: dataset_with_noise.csv
